In [ ]:
%pip install imblearn

## expand a labeled dataset to SMOTE

magnify the number of minority/exceptional cases within the sequence dataset, ideally targets the binary binned dataset.

[reference 1](<ver5-ordinal-binning-grid-searches/step 2-0, ranged clustering, with time.ipynb>)

different oversampling tools: Naive random oversampling, SMOTE, ADASYN, SMOTENC

In [ ]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE
from collections import Counter
import pandas as pd 

In [ ]:
m2_pipeline = pd.read_csv("binary_binned_pipeline1.csv")

In [ ]:
m2_pipeline.columns #do this to identify the index of the categorical feature, for below setup

In [ ]:
#set up toolsets as functions to build separate datasets, bin_Naive, bin_SMOTE, bin_ADASYN

def build_naive(d):  #https://imbalanced-learn.org/stable/over_sampling.html#naive-random-over-sampling
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X, y)
    print("ROS",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_smote(d): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html#smote
    X_resampled, y_resampled = SMOTE(random_state=42,categorical_features='label', categorical_encoder=None, ).fit_resample(X, y)
    print("SMOTE",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_adasyn(d): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.ADASYN.html#adasyn
    X_resampled, y_resampled = ADASYN(random_state=42).fit_resample(X, y)
    print("ADASYN",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_borderline(d): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.BorderlineSMOTE.html#borderlinesmote
    X_resampled, y_resampled = BorderlineSMOTE(random_state=42).fit_resample(X, y)
    print("BORDERLINE",sorted(Counter(y_resampled).items())) 
    return X_resampled, y_resampled

def build_smotenc(d): #https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTENC.html#smotenc
    smote_nc = SMOTENC(categorical_features=[0, 2], random_state=42)
    X_resampled, y_resampled = smote_nc.fit_resample(X, y)
    print("SMOTENC",sorted(Counter(y_resampled).items()))
    return X_resampled, y_resampled

def build_svmsmote(d):
    sm = SVMSMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X, y)
    print("SVMSMOTE",'Resampled dataset shape %s' % Counter(y_res))
    return X_res, y_res

def build_kmsmote(d):
    m = KMeansSMOTE(kmeans_estimator=MiniBatchKMeans(n_init=1, random_state=0), random_state=42)
    X_res, y_res = sm.fit_resample(X, y)
    # Find the number of new samples in the middle blob
    n_res_in_middle = ((X_res[:, 0] > -5) & (X_res[:, 0] < 5)).sum()
    print("KMSMOTE","Samples in the middle blob: %s" % n_res_in_middle)
    return X_res, y_res

In [ ]:
#loop through each oversampled dataset, then run the classifier search on that set, outlining each set, before hand

#for each set, run the classifier search
source = 'binary_binned_pipeline.csv'
resultSet = []
resultSet.append(getBestClassifier('naive', build_naive(source)))
resultSet.append(getBestClassifier('smote', build_smote(source)))
resultSet.append(getBestClassifier('adasyn', build_adasyn(source)))
resultSet.append(getBestClassifier('borderline', build_borderline(source)))
resultSet.append(getBestClassifier('smotenc', build_smotenc(source)))
resultSet.append(getBestClassifier('svmsmote', build_svmsmote(source)))
resultSet.append(getBestClassifier('kmsmote', build_svmsmote(source)))
resultDF = pd.DataFrame(resultSet)

In [ ]:
resultDF

### take optimal classifier parameters and technique